In [1]:
from IPython.display import Audio, display
import numpy as np
import time
import librosa
import librosa.display
# import soundfile as sf

# Parameter

In [2]:
input_file_name = 'rauschen.wav'
output_file_name = 'keinRauschen.wav'

frame_size = 1024  # Größe eines Frames
hop_size = 512  # Schrittweite zwischen den Frames
threshold = 0.5  # Schwellenwert für die Filterung

# FFT und DFT Implemtierung

In [3]:
# Berechne die Faktoren fuer die Kombination
EXP_FACTOR_FFT_LOOK_UP = [np.exp(-2j * np.pi * np.arange(N // 2) / N) for N in range(frame_size + 1)]
EXP_FACTOR_IFFT_LOOK_UP = [np.exp(2j * np.pi * np.arange(N // 2) / N) for N in range(frame_size + 1)]

# Rekursive FFT Implementierung für 1D Arrays
def fft(x):
    N = x.shape[0]  # N ist die Laenge von x
    
    # Falls x nur aus einem Element besteht gebe x zurueck
    if N == 1: return x

    # Teile a in gerade und ungerade Elemente und berechne deren FFT
    fft_even = fft(x[0::2])
    fft_odd = fft(x[1::2])

    # Führe die Butterfly Operation durch
    exp_mult_fft_odd = EXP_FACTOR_FFT_LOOK_UP[N] * fft_odd
    butterfly = np.concatenate([fft_even + exp_mult_fft_odd, fft_even - exp_mult_fft_odd])
        
    return butterfly

# Rekursive Idft Implementierung für 1D Arrays
def ifft(x):
    N = x.shape[0]  # N ist die Länge von x
    
    # Falls x nur aus einem Element besteht, gebe x zurück
    if N == 1: return x

    # Teile x in gerade und ungerade Elemente und berechne deren IFFT
    ifft_even = ifft(x[0::2])
    ifft_odd = ifft(x[1::2])
    
    # Führe die Butterfly Operation durch
    exp_mult_ifft_odd = EXP_FACTOR_IFFT_LOOK_UP[N] * ifft_odd
    butterfly = np.concatenate([ifft_even + exp_mult_ifft_odd, ifft_even - exp_mult_ifft_odd])
    
    # Am Ende der IFFT muss jede Komponente durch N dividiert werden
    return butterfly / N


# Direkte Implementierung der DFT für 1D Arrays
def dft(x):
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    e = np.exp(-2j * np.pi * k * n / N)
    return np.dot(e, x)

# Direkte Implementierung der IDFT für 1D Arrays
def idft(x):
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    e = np.exp(2j * np.pi * k * n / N)
    return np.dot(e, x) / N

# Filter methoden

In [4]:
# Funktion zur Anwendung der FFT und Filterung
def filter_frame_fft(frame, threshold):
    # Anwendung der FFT
    spectrum = fft(frame)
    # Filterung: Abschwächung von Frequenzen basierend auf einem Schwellenwert
    spectrum[np.abs(spectrum) < threshold] = 0
    # Anwendung der inversen FFT
    filtered_frame = ifft(spectrum)
    return filtered_frame.real

# Funktion zur Anwendung der FFT und Filterung
def filter_frame_dft(frame, threshold):
    # Anwendung der FFT
    spectrum = dft(frame)
    # Filterung: Abschwächung von Frequenzen basierend auf einem Schwellenwert
    spectrum[np.abs(spectrum) < threshold] = 0
    # Anwendung der inversen FFT
    filtered_frame = idft(spectrum)
    return filtered_frame.real

# Berechnung mittels FFT Algorithmus

In [5]:
# Audiosignal laden
signal, sample_rate = librosa.load(input_file_name, sr=None)

start_time = time.time()
# Aufteilung des Signals in Frames und Filterung
filtered_signal = np.array([])
for i in range(0, len(signal) - frame_size, hop_size):
    frame = signal[i:i+frame_size]
    filtered_frame = filter_frame_fft(frame, threshold)
    # Überlappende Addition für die Rekonstruktion
    if filtered_signal.size == 0:
        filtered_signal = filtered_frame
    else:
        filtered_signal[-hop_size:] += filtered_frame[:hop_size]
        filtered_signal = np.append(filtered_signal, filtered_frame[hop_size:])

# time tracking
duration = time.time() - start_time

print("Duration FFT: ", duration, "s")

# Ergebnis speichern
# sf.write(output_file_name, filtered_signal, sample_rate)

Duration FFT:  3.8194119930267334


# Play Audio

In [11]:
# Create audio widgets
print("Original Audio:")
display(Audio(signal, rate=sample_rate, autoplay=False, normalize=True))
print("Processed Audio:")
display(Audio(filtered_signal, rate=sample_rate, autoplay=False, normalize=True))

Original Audio:


Processed Audio:


# Berechnung mittels DFT Algorithmus

In [ ]:
# Audiosignal laden
signal, sample_rate = librosa.load(input_file_name, sr=None)

start_time = time.time()
# Aufteilung des Signals in Frames und Filterung
filtered_signal = np.array([])
for i in range(0, len(signal) - frame_size, hop_size):
    frame = signal[i:i+frame_size]
    filtered_frame = filter_frame_dft(frame, threshold)
    # Überlappende Addition für die Rekonstruktion
    if filtered_signal.size == 0:
        filtered_signal = filtered_frame
    else:
        filtered_signal[-hop_size:] += filtered_frame[:hop_size]
        filtered_signal = np.append(filtered_signal, filtered_frame[hop_size:])

#time tracking
duration = time.time() - start_time

print("Duration DFT: ", duration)

# Ergebnis speichern
# sf.write(output_file_name, filtered_signal, sample_rate)

# Play Audio

In [10]:
# Create audio widgets
print("Original Audio:")
display(Audio(signal, rate=sample_rate, autoplay=False, normalize=True))
print("Processed Audio:")
display(Audio(filtered_signal, rate=sample_rate, autoplay=False, normalize=True))

Original Audio:


Processed Audio:
